# Hessian-based uncertainty quantification

**Author:** James R. Maddison

This notebook describes the use of tlm_adjoint for Hessian-based uncertainty quantification. The approach is based on the method described in

- Tobin Isaac, Noemi Petra, Georg Stadler, and Omar Ghattas, 'Scalable and efficient algorithms for the propagation of uncertainty from data through inference to prediction for large-scale problems, with application to flow of the Antarctic ice sheet', Journal of Computational Physics, 296, pp. 348&ndash;368, 2015, doi: 10.1016/j.jcp.2015.04.047

We assume real spaces and a real build of Firedrake throughout.

## Uncertainty quantification problem

We consider the advection-diffusion equation in the unit square domain

$$\partial_t u + \nabla^\perp m \cdot \nabla u = \kappa \nabla^2 u,$$

for some real and positive $\kappa$ and subject to doubly periodic boundary conditions. We consider a continuous Galerkin finite element discretization in space and an implicit midpoint rule discretization in time, seeking $u_{n + 1} \in V$ such that

$$\forall \zeta \in V \qquad \int_\Omega \zeta u_{n + 1} + \Delta t \int_\Omega \zeta \nabla^\perp m \cdot \nabla \left[ \frac{1}{2} ( u_n + u_{n + 1}) \right]
    + \kappa \Delta t \int_\Omega \nabla \zeta \cdot \nabla \left[ \frac{1}{2} ( u_n + u_{n + 1}) \right] = \int_\Omega \zeta u_n,$$

with timestep size $\Delta t$ and given some suitable discrete initial condition $u_0 \in V$. Here $V$ is a real continuous $P_1$ finite element space whose elements satisfy the doubly periodic boundary conditions, and after discretizing we have $m \in V$. For a given value of $m$ we let $\hat{u}_N ( m )$ denote the solution obtained after $N$ timesteps. Given an observation for $\hat{u}_N ( t )$, $u_{obs} \in V$, we seek to infer information about the control $m$. Here we are specifically seeking to infer the transport, defined in terms of a stream function $m$, which transports the solution from $u_0$ to $u_{obs}$ in time $T$.

We first need to model the observational error. For a given value of the control $m$ we treat observations are being realizations of a Gaussian random variable, with mean given by $\hat{u}_N ( m )$, and with known covariance. Specifically we define a density $p ( u_{obs} | m )$ whose negative logarithm is (up to a normalization term which is neglected here)

$$-\ln p ( u_{obs} | m ) = \frac{1}{2} R_{obs}^{-1} ( u_{obs} - \hat{u}_N ( m ), u_{obs} - \hat{u}_N ( m ) ).$$

$R_{obs}^{-1}$ is a bilinear and symmetric positive definite observational inverse covariance operator. Here we choose $R_{obs}^{-1}$ by defining

$$R_{obs}^{-1} ( q_i, q_j ) = \int_\Omega q_j \mathcal{L}_{\sigma_R,d_R}^2 ( q_i ),$$

where $\mathcal{L}_{\sigma,d} : V \rightarrow V$ is defined by

$$\forall \zeta \in V \qquad \frac{1}{\sqrt{4 \pi \sigma^2}} \left[ \frac{1}{d} \int_\Omega \zeta q + d \int_\Omega \nabla \zeta \cdot \nabla q \right] = \int_\Omega \zeta \mathcal{L}_{\sigma,d} \left( q \right).$$

In the continuous and $\Omega = \mathbb{R}^2$ case $R_{obs}$ then defines a covariance operator with single point variance $\sigma_R^2$ and autocorrelation length scale $d_R$ (see Lindgren et al 2011, doi: 10.1111/j.1467-9868.2011.00777.x).

We next introduce a prior for the control. We consider a Gaussian prior with mean zero and with known covariance. Specifically we define a prior density $p ( m )$ whose negative logarithm is (again up to a normalization term)

$$-\ln p ( m ) = \frac{1}{2} B^{-1} ( m, m ).$$

$B^{-1}$ is a bilinear and symmetric positive definite observational inverse covariance operator. Here we choose $B^{-1}$ by defining

$$B^{-1} ( q_i, q_j ) = \int_\Omega q_j \mathcal{L}_{\sigma_B,d_B}^2 ( q_i ).$$

Now applying Bayes theorem we obtain a posterior density whose negative logarithm is (up to a normalization term)

$$-\ln p ( m | u_{obs} ) = \frac{1}{2} R_{obs}^{-1} ( u_{obs} - \hat{u}_N ( m ), u_{obs} - \hat{u}_N ( m ) ) + \frac{1}{2} B^{-1} ( m, m ).$$

We have made a number of modelling choices, but subject to these choices the posterior density now completely describes the information we have about the control after being supplied with an observation $u_{obs}$. The challenge is that the posterior is defined in a high dimensional space, and is in general not Gaussian. To simplify the problem we *approximate* the posterior with a Gaussian, and specifically make the approximation (up to a normalization term)

$$-\ln p ( m | u_{obs} ) \approx \frac{1}{2} \Gamma_{post}^{-1} ( m - m_{MAP}, m - m_{MAP} ).$$

The mean of the Gaussian approximation is set equal to the posterior density maximizer (the Maximum A Posteriori estimate), $m_{map}$. The inverse covariance of the Gaussian approximation, $\Gamma_{post}^{-1}$, is set equal to the Hessian, $H$, of the negative log posterior density $-\ln p ( m | u_{obs} )$, defined by differentiating twice with respect to $m$ and evaluated at the posterior density maximizer.

Unfortunately in order to quantify uncertainty we require information about the *covariance*, and not the *inverse covariance*. Specifically if we have a linear observational operator $q \in V^*$ then our estimate for the variance associated with $q ( m )$ is

$$\sigma^2_q \approx \Gamma_{post} ( q, q ) = H^{-1} ( q, q ),$$

which requires access to information about the Hessian *inverse*.

In the following we use two approaches to gain access to this information.

1. A low rank update approximation. We approximate the Hessian inverse using a low rank update to the prior covariance, using the methodology of Isaac et al 2015, doi: 10.1016/j.jcp.2015.04.047.
2. By computing the Hessian inverse action using a Krylov method, preconditioned using the low rank update approximation.

To solve this problem we need several pieces:

1. A differentiable solver for the forward problem. We will construct this using Firedrake with tlm_adjoint.
2. To find the posterior maximizer. We will use gradient-based optimization using TAO.
3. To find a low rank update approximation for the Hessian inverse. We will find this using a partial eigenspectrum obtained using SLEPc.
4. To compute the Hessian inverse action. We will compute this using a Krylov method using PETSc, preconditioned using the partial eigenspectrum.

## Forward problem

We first implement the forward model using Firedrake with tlm_adjoint.

In [ ]:
%matplotlib inline

from firedrake import *
from tlm_adjoint.firedrake import *
from firedrake.pyplot import tricontourf

import matplotlib.pyplot as plt
import numpy as np

reset_manager()
clear_caches()

mesh = PeriodicUnitSquareMesh(40, 40, diagonal="crossed")
X = SpatialCoordinate(mesh)
space = FunctionSpace(mesh, "Lagrange", 1)
test = TestFunction(space)
trial = TrialFunction(space)

T = 0.5
N = 10
kappa = Constant(1.0e-3, static=True)
dt = Constant(T / N, static=True)
sigma_R = Constant(0.03)
d_R = Constant(0.1)
sigma_B = Constant(0.025)
d_B = Constant(0.2)
u_0 = Function(space, name="u_0").interpolate(
    exp(-((X[0] - 0.4) ** 2 + (X[1] - 0.5) ** 2) / (2 * 0.08 * 0.08)))
u_obs = Function(space, name="u_obs").interpolate(
    exp(-((X[0] - 0.6) ** 2 + (X[1] - 0.5) ** 2) / (2 * 0.08 * 0.08)))


def L(sigma, d, q):
    u = Function(space)
    solve(inner(trial, test) * dx
          == (1.0 / sqrt(4.0 * np.pi * sigma * sigma))
          * ((1.0 / d) * inner(q, test) * dx + d * inner(grad(q), grad(test)) * dx),
          u, solver_parameters={"ksp_type": "preonly",
                                "pc_type": "cholesky"})
    return u


def L_inv(sigma, d, q):
    u = Function(space)
    solve((1.0 / sqrt(4.0 * np.pi * sigma * sigma))
          * ((1.0 / d) * inner(trial, test) * dx + d * inner(grad(trial), grad(test)) * dx)
          == inner(q, test) * dx,
          u, solver_parameters={"ksp_type": "preonly",
                                "pc_type": "cholesky"})
    return u


def R_inv_term(q):
    L_q = L(sigma_R, d_R, q)
    LL_q = L(sigma_R, d_R, L_q)
    return Functional(name="J_mismatch").assign(0.5 * inner(LL_q, q) * dx)


def R_inv_action(q):
    L_q = L(sigma_R, d_R, q)
    LL_q = L(sigma_R, d_R, L_q)
    return assemble(inner(LL_q, test) * dx)


def B_inv_term(q):
    L_q = L(sigma_B, d_B, q)
    LL_q = L(sigma_B, d_B, L_q)
    return Functional(name="J_prior").assign(0.5 * inner(LL_q, q) * dx)


def B_inv_action(q):
    L_q = L(sigma_B, d_B, q)
    LL_q = L(sigma_B, d_B, L_q)
    return assemble(inner(LL_q, test) * dx)


def B_action(q):
    u = Function(space)
    solve(inner(trial, test) * dx == q,
          u, solver_parameters={"ksp_type": "preonly",
                                "pc_type": "cholesky"})
    Li_q = L_inv(sigma_B, d_B, u)
    LiLi_q = L_inv(sigma_B, d_B, Li_q)
    return LiLi_q


def forward(m):
    m = Function(space, cache=True).assign(m)
    u_np1 = Function(space, name="u_np1")
    u_n = u_0.copy(deepcopy=True)
    eq = EquationSolver(inner(trial, test) * dx
                        + 0.5 * dt * inner(dot(perp(grad(m)), grad(trial)), test) * dx
                        + 0.5 * kappa * dt * inner(grad(trial), grad(test)) * dx
                        == inner(u_n, test) * dx
                        - 0.5 * dt * inner(dot(perp(grad(m)), grad(u_n)), test) * dx
                        - 0.5 * kappa * dt * inner(grad(u_n), grad(test)) * dx,
                        u_np1, solver_parameters={"ksp_type": "preonly",
                                                  "pc_type": "lu"})
    for _ in range(N):
        eq.solve()
        u_n.assign(u_np1)

    u_mismatch = Function(space, name="u_mismatch").assign(u_n - u_obs)
    J_mismatch = R_inv_term(u_mismatch)
    J_prior = B_inv_term(m)
    return J_mismatch, J_prior, J_mismatch + J_prior, u_np1

Let's first visualize the initial condition $u_0$ and the observation $u_{obs}$ taken a time $T$ later.

In [ ]:
def plot_output(u, title):
    r = (u.dat.data_ro.min(), u.dat.data_ro.max())
    eps = (r[1] - r[0]) * 1.0e-12
    p = tricontourf(u, np.linspace(r[0] - eps, r[1] + eps, 32))
    plt.gca().set_title(title)
    plt.colorbar(p)
    plt.gca().set_aspect(1.0)


plot_output(u_0, r"$u_0$")
plot_output(u_obs, r"$u_{obs}$")

## Quantities of interest

In the following discussion we seek to quantify uncertainties in *quantities of interest*. These are defined using linear functionals, say $q \in V^*$. Given some value of the control $m$, $q ( m )$ is the value of the quantity of interest. Moreover, given the Maximum A Posteriori estimate $m_{map}$ for the control (the posterior density maximizer), the estimate for the posterior mean of the quantity of interest is given by

$$\mu_{q,posterior} \approx q ( m_{map} ).$$

To obtain uncertainty estimates we need a covariance operator. Note that a covariance operator, say $\Gamma_{post}$, is a bilinear operator, $\Gamma_{post} : V^* \times V^* \rightarrow \mathbb{R}$. It takes the linear functionals we use to obtain values for two quantities of interest, and gives us a value for a covariance between them. In particular the estimate for the posterior uncertainty in a single quantity of interest, defined by a linear functional $q \in V^*$, is

$$\sigma_{q,posterior} \approx \sqrt{ \Gamma_{post} ( q, q ) } = \sqrt{ H^{-1} ( q, q ) }.$$

## Finding the posterior maximizer

We now find the posterior density maximizer, which corresponds to seeking a minimum of the negative log posterior density. We solve this problem using the Toolkit for Advanced Optimization (TAO), with the Limited-Memory Variable Metric (LMVM) approach. This is a gradient-based approach, so let's first verify the adjoint using Taylor verification.

In [ ]:
m_0 = Function(space, name="m_0").interpolate(
    0.06 * sin(2 * pi * X[1]))

reset_manager()
start_manager()
_, _, J, _ = forward(m_0)
stop_manager()

dJ = compute_gradient(J, m_0)
dm = Function(space, name="dm").interpolate(
    sin(4 * pi * X[0]) * cos(6 * pi * X[1]))
min_order = taylor_test(lambda m: forward(m)[2], m_0, J_val=J.value, dJ=dJ, dM=dm, seed=1.0e-6)
assert min_order > 1.99

Next we solve the optimization problem. The considered problem seeks to infer the transport in the advection-diffusion equation, in terms of a stream function. Initially the tracer is concentrated on the left, and the observation taken a time $T$ later has the tracer moved to the right. The variable `m_0`, which will define our initial guess for the optimization, is set so that the velocity at the center has approximately the correct magnitude for this transport.

As usual we need to define an appropriate inner product associated with derivatives. Here the prior defines a natural inner product &ndash; specifically we can use the prior covariance, $B$, to define an inner product for the dual space $V^*$.

In [ ]:
optimizer = TAOSolver(lambda m: forward(m)[2], space, H_0_action=B_action,
                      solver_parameters={"tao_type": "lmvm",
                                         "tao_gatol": 1.0e-5,
                                         "tao_grtol": 0.0,
                                         "tao_gttol": 0.0,
                                         "tao_monitor": None})

m_map = Function(space, name="m_map").assign(m_0)
optimizer.solve(m_map)

reset_manager()
start_manager()
J_mismatch, _, J, u_map = forward(m_map)
stop_manager()

Let's plot the result.

In [ ]:
reset_manager()
start_manager()
J_mismatch, _, J, u_map = forward(m_map)
stop_manager()

plot_output(m_map, r"$m_{map}$")
plot_output(u_map, r"$\hat{u} ( m_{map} )$")

## Computing uncertainty estimates: Low rank update approximation

We next seek to use Hessian information to quantify the uncertainty in the result of the inference. We use a low-rank update approximation using the methodology of Isaac et al 2015, doi: 10.1016/j.jcp.2015.04.047.

In this approach we consider the mismatch Hessian, $H_{mismatch}$, obtained by differentiating

$$J_{mismatch} = \frac{1}{2} R_{obs}^{-1} ( u_{obs} - \hat{u}_N ( m ), u_{obs} - \hat{u}_N ( m ) ),$$

twice with respect to the control $m$. i.e.

$$H_{mismatch} = H - B^{-1}.$$

We seek eigenpairs $\lambda_k \in \mathbb{R}$, $v_k \in V$ such that

$$H_{mismatch} \left( v_k, \cdot \right) = \lambda_k B^{-1} \left( v_k, \cdot \right),$$

where the eigenvectors are $B^{-1}$-orthonormal

$$B^{-1} ( v_k, v_l ) = \delta_{k,l}.$$

This leads directly to an expression for a low-rank update approximation for the Hessian inverse, expressed as a low rank update to the prior covariance (equation (20) of Isaac et al 2015, doi: 10.1016/j.jcp.2015.04.047),

$$H^{-1} ( q_i, q_j ) \approx B ( q_i, q_j ) - \sum_k \frac{\lambda_k}{1 + \lambda_k} q_i ( v_k ) q_j ( v_k ),$$

which we use to approximate the posterior covariance.

We first perform a higher order Taylor verification using the mismatch Hessian.

In [ ]:
H_mismatch = CachedHessian(J_mismatch)

dm = Function(space, name="dm").interpolate(
    exp(-((X[0] - 0.3) ** 2 + (X[1] - 0.3) ** 2) / (2 * 0.08 * 0.08)))
min_order = taylor_test(lambda m: forward(m)[0], m_map, J_val=J_mismatch.value, ddJ=H_mismatch, dM=dm, seed=1.0e-4)
assert min_order > 2.99

We now use SLEPc, seeking the 20 eigenpairs whose eigenvalues have largest magnitude.

Note: There is a notational clash here! Conventially the prior inverse covariance is denoted $B^{-1}$, but here we use this on the right-hand-side of a generalized eigenproblem, where the associated matrix is *also* conventially notated $B$. Here the `B_action` argument to the `HessianEigensolver` constructor is set equal to `B_inv_action` in the call, and the `B_inv_action` argument is set equal to `B_action`!

In [ ]:
esolver = HessianEigensolver(
    H_mismatch, m_map, B_action=B_inv_action, B_inv_action=B_action,
    solver_parameters={"eps_type": "krylovschur",
                       "eps_gen_hermitian": None,
                       "eps_largest_magnitude": None,
                       "eps_nev": 20,
                       "eps_conv_rel": None,
                       "eps_tol": 1.0e-14,
                       "eps_purify": False,
                       "eps_monitor": None})
esolver.solve()

Let's plot the eigenvalues.

In [ ]:
lam = esolver.eigenvalues()
assert (lam > 0.0).all()
plt.semilogy(range(1, len(esolver) + 1), lam, "k-")
plt.xlim(0, len(esolver))
plt.xlabel("Eigenvalue index")
plt.ylabel("Eigenvalue")

Each eigenvalue indicates some information, added by the observation, over the prior. Specifically each eigenvector $v_k \in V$ has an associated dual space element defined by the prior inverse covariance, $q_{v_k} = B^{-1} ( v_k, \cdot ) \in V^*$. If we have an observation for a quantitity of interest defined by a linear functional equal to a (non-zero) multiple of this associated dual space element, and if the posterior were Gaussian, then the associated variance decreases by a factor of one plus the eigenvalue when we add the observation. That is, under the Gaussian approximation, we have

$$\frac{\sigma^2_{q_{v_k},posterior}}{\sigma^2_{q_{v_k},prior}} = \frac{1}{1 + \lambda_k},$$

where $\sigma^2_{q_{v_k},posterior}$ and $\sigma^2_{q_{v_k},prior}$ are respectively the posterior and prior variance associated with $q_{v_k}$, and where $\lambda_k$ is the associated eigenvalue.

Having retained only $20$ eigenpairs, the smallest eigenvalue obtained is quite a bit bigger than one. This means we might expect to be missing quite a bit of information available in the Hessian if we use a low rank update approximation using only these $20$ eigenpairs. We'll return to this issue later.

The linear functional associated with computing the average $x$-component of the velocity for $0.4 < y < 0.6$, $q_u \in V^*$, satisfies

$$q_u ( \zeta ) = -\frac{1}{\int_\Omega \mathcal{I}} \int_\Omega \mathcal{I} \partial_y \zeta,$$

where $\mathcal{I}$ is one where $0.4 < y < 0.6$ and zero elsewhere. We can use this to evaluate an estimate for the posterior mean of this quantity,

$$\mu_{q_u,posterior} \approx q_u ( m_{map} ).$$

In [ ]:
I = Function(FunctionSpace(mesh, "Discontinuous Lagrange", 0)).interpolate(
    conditional(X[1] > 0.4, 1.0, 0.0) * conditional(X[1] < 0.6, 1.0, 0.0))
q_u = assemble(-(1 / Constant(assemble(I * dx))) * I * test.dx(1) * dx)
print(f"{assemble(q_u(m_map))=}")

The associated posterior uncertainty estimate is then

$$\sigma_{q_u,posterior} \approx \sqrt{H_{approx}^{-1} ( q_u, q_u )}.$$

In [ ]:
sigma_u = sqrt(assemble(q_u(esolver.spectral_approximation_solve(q_u))))
print(f"{sigma_u=}")

We can compare this with the prior uncertainty,

$$\sigma_{q_u,prior} = \sqrt{ B ( q_u, q_u ) }.$$

In [ ]:
sigma_prior_q_u = sqrt(assemble(q_u(B_action(q_u))))
print(f"{sigma_prior_q_u=}")

i.e. we estimate that the observation has reduced the uncertainty in our estimate for $q_u( m )$ by about 25%.

## Computing uncertainty estimates: Full Hessian inverse

We have made a number of assumptions in order to define the inference problem, but given the problem we have used only two approximations in the uncertainty estimate itself: the local Gaussian assumption (making use of the Hessian), and the low rank update approximation for the Hessian inverse. We now remove the second of these approximations using a Krylov solver, using PETSc. Since we already have an *approximation* for the Hessian inverse, we can use this to construct a preconditioner in the calculation of a Hessian inverse action.

In [ ]:
H = CachedHessian(J)

ksp = HessianLinearSolver(H, m_map, solver_parameters={"ksp_type": "cg",
                                                       "ksp_atol": 0.0,
                                                       "ksp_rtol": 1.0e-7,
                                                       "ksp_monitor": None},
                          pc_fn=esolver.spectral_pc_fn())
H_inv_q_u = Function(space, name="H_inv_q_u")
ksp.solve(H_inv_q_u, q_u)

Let's double check that we solved the linear system.

In [ ]:
residual = Cofunction(space.dual()).assign(
    H.action(m_map, H_inv_q_u)[2] - q_u)
q_u_norm = abs(q_u.dat.data_ro).max()
residual_norm = abs(residual.dat.data_ro).max()
print(f"{residual_norm / q_u_norm=}")
assert residual_norm / q_u_norm < 1.0e-4

Our new uncertainty estimate, using the full Hessian, is

In [ ]:
sigma_u = sqrt(assemble(q_u(H_inv_q_u)))
print(f"{sigma_u=}")

This reveals that our partial eigenspectrum estimate did indeed miss quite a bit of relevant information available in the Hessian. In fact with the full Hessian we estimate a reduction in uncertainty of about 55%.